In [6]:
# First cell - Imports
%load_ext autoreload
%autoreload 2

# Python Libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
from scipy import stats
from IPython.display import display

#Modules
from fred_loader import fred_load
from fred_transformer import fred_transform
from fred_visualizer import fred_visualize

In [7]:
# Cell 2: Load Data
df = fred_load()
print("Data loaded successfully!")
#Verify data loaded
display(df.head())

Data loaded successfully!


,yield_spread,gdp,fed_funds,unemployment,option_adjusted_spread,delinquency_rate_credit_cards,delinquency_rate_loans,cpi,pce
date,,,,,,,,,
1976-01-01,NaN,6323.649,5.37,7.9,NaN,NaN,NaN,55.8,27.795
1976-01-02,NaN,NaN,5.28,NaN,NaN,NaN,NaN,NaN,NaN
1976-01-03,NaN,NaN,5.28,NaN,NaN,NaN,NaN,NaN,NaN
1976-01-04,NaN,NaN,5.28,NaN,NaN,NaN,NaN,NaN,NaN
1976-01-05,NaN,NaN,5.29,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# Cell 3: Transform Data
df = fred_transform(df)
print("Transformation complete!")
display(df.head())
# Print to Excel
df.to_excel('stats.xlsx')

Transformation complete!


c:\Users\mrasm\econ_indicator_analysis\fred_transformer.py:9: FutureWarning:

The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.



,yield_spread,gdp,fed_funds,unemployment,option_adjusted_spread,delinquency_rate_credit_cards,delinquency_rate_loans,cpi,pce,gdp_growth,quarterly_spread,next_quarter_delinquency,quarter,period
date,,,,,,,,,,,,,,
1976-01-01,NaN,6323.649,5.37,7.9,NaN,NaN,NaN,55.8,27.795,NaN,NaN,NaN,1Q76,Expansion
1976-01-02,NaN,NaN,5.28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1Q76,Expansion
1976-01-03,NaN,NaN,5.28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1Q76,Expansion
1976-01-04,NaN,NaN,5.28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1Q76,Expansion
1976-01-05,NaN,NaN,5.29,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1Q76,Expansion


In [4]:
# Fourth Cell - Data Analysis (Two Variables)
# Create new dataframe with only necessary data
df_viz = df.reset_index().copy()
df_viz = df_viz[df_viz['date'] >= '1996-12-31']
df_viz = df_viz[['quarter', 'period', 'quarterly_spread', 'delinquency_rate_loans']]
df_viz = df_viz.drop_duplicates()
spread_loan_correlation = df_viz[['quarterly_spread', 'delinquency_rate_loans']].corr()
correlation_coefficient, p_value = stats.pearsonr(df_viz['quarterly_spread'], df_viz['delinquency_rate_loans'])
print(f"P-value: {p_value:.3e}")  # Us
print("Correlation between Option-Adjusted Spread and delinquency rates:")


def plot_loan_spread(df_viz):
    # Calculate correlation
    spread_loan_correlation = df_viz['quarterly_spread'].corr(df_viz['delinquency_rate_loans'])
    # Create the base scatter plot
    fig = px.scatter(
        df_viz,
        x='quarterly_spread',
        y='delinquency_rate_loans',
        color='period',
        trendline="ols",
        color_discrete_map={
            'Expansion': '#2E86C1',      # Rich blue
            'Dot Com': '#E74C3C',        # Crimson red
            'Great Recession': '#8E44AD', # Deep purple
            'COVID': '#F39C12'           # Golden orange
        }
    )

    # Update traces for better visibility
    fig.update_traces(
        marker=dict(
            size=12,
            line=dict(width=1, color='white')
        ),
        selector=dict(mode='markers')
    )

    # Style the trendline
    fig.update_traces(
        line=dict(width=2, dash='dot'),
        selector=dict(mode='lines')
    )

    # Enhanced layout
    fig.update_layout(
        title=dict(
            text=f'Corporate Loan Risk Analysis<br><span style="font-size: 14px">Delinquency Rate vs Option-Adjusted Spread (Correlation: {spread_loan_correlation:.3f})</span>',
            font=dict(size=24),
            x=0.5,
            y=0.95
        ),
        plot_bgcolor='white',
        paper_bgcolor='white',
        font=dict(
            family="Arial",
            size=12
        ),
        legend=dict(
            title="Economic Period",
            borderwidth=1,
            bordercolor='#E5E5E5',
            bgcolor='rgba(255, 255, 255, 0.9)',
            x=0.02,
            y=0.98
        ),
        width=1000,
        height=600,
        margin=dict(t=100, l=80, r=40, b=80)
    )

    # Enhance axes
    fig.update_xaxes(
        title=dict(
            text="Option-Adjusted Spread (basis points)",
            font=dict(size=14)
        ),
        showgrid=True,
        gridwidth=1,
        gridcolor='#E5E5E5',
        mirror=True,
        showline=True,
        linewidth=2,
        linecolor='#2C3E50'
    )

    fig.update_yaxes(
        title=dict(
            text="Delinquency Rate (%)",
            font=dict(size=14)
        ),
        showgrid=True,
        gridwidth=1,
        gridcolor='#E5E5E5',
        mirror=True,
        showline=True,
        linewidth=2,
        linecolor='#2C3E50'
    )

    # FRED Source
    fig.add_annotation(
        text="Source: Federal Reserve Economic Data (FRED)",
        xref="paper", yref="paper",
        x=0.0000001, y=-0.15,
        showarrow=False,
        font=dict(size=9, color='gray')
    )

    # Relationship caption
    fig.add_annotation(
        text="Higher spreads generally indicate increased market stress and correlate with higher delinquency rates",
        xref="paper", yref="paper",
        x=0.8, y=-0.15,
        showarrow=False,
        font=dict(size=10, color='gray')
    )

    return fig

plot_loan_spread(df_viz)

P-value: 8.960e-13
Correlation between Option-Adjusted Spread and delinquency rates:


In [5]:
##AI - Generated PDF Code for testing - delete

import plotly.subplots as sp

def create_sample_data():
    """Create sample dataset for visualization"""
    np.random.seed(42)
    dates = pd.date_range(start='2023-01-01', end='2023-12-31', freq='D')

    data = pd.DataFrame({
        'date': dates,
        'sales': np.random.normal(100, 15, len(dates)) + np.linspace(0, 30, len(dates)),
        'temperature': np.random.normal(20, 5, len(dates)) + 10 * np.sin(np.linspace(0, 2*np.pi, len(dates))),
        'category': np.random.choice(['A', 'B', 'C', 'D'], len(dates)),
    })

    return data

def create_summary_stats(data):
    """Create summary statistics for the table"""
    monthly_stats = data.set_index('date').resample('M').agg({
        'sales': ['mean', 'std', 'min', 'max'],
        'temperature': 'mean'
    }).round(2)

    monthly_stats.columns = ['Avg Sales', 'Sales StdDev', 'Min Sales', 'Max Sales', 'Avg Temp']
    monthly_stats.index = monthly_stats.index.strftime('%B')
    return monthly_stats

def create_dashboard(data):
    """Create a multi-plot dashboard with table"""
    # Create figure with secondary y-axis
    fig = sp.make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Daily Sales Trend',
            'Sales by Category',
            'Monthly Summary Statistics',
            'Monthly Sales Distribution'
        ),
        vertical_spacing=0.2,  # Increase spacing between rows
        horizontal_spacing=0.1,  # Adjust horizontal spacing
        specs=[[{"secondary_y": True}, {}],
               [{"type": "table"}, {}]]
    )

    # 1. Line plot with moving average
    ma = data['sales'].rolling(window=7).mean()
    fig.add_trace(
        go.Scatter(x=data['date'], y=data['sales'], name="Daily Sales",
                  line=dict(color='#1f77b4', width=1)),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(x=data['date'], y=ma, name="7-day Moving Average",
                  line=dict(color='#ff7f0e', width=2)),
        row=1, col=1
    )

    # 2. Bar chart
    category_sales = data.groupby('category')['sales'].mean().reset_index()
    fig.add_trace(
        go.Bar(x=category_sales['category'], y=category_sales['sales'],
               name="Category Sales", marker_color='#2ca02c'),
        row=1, col=2
    )

    # 3. Summary Statistics Table
    monthly_stats = create_summary_stats(data)
    fig.add_trace(
        go.Table(
            header=dict(
                values=['Month'] + list(monthly_stats.columns),
                fill_color='#2c3e50',
                align='left',
                font=dict(color='white', size=12)
            ),
            cells=dict(
                values=[monthly_stats.index] + [monthly_stats[col] for col in monthly_stats.columns],
                fill_color='#f9f9f9',
                align=['left', 'right', 'right', 'right', 'right', 'right'],
                font=dict(color=['#2c3e50'], size=11)
            )
        ),
        row=2, col=1
    )

    # 4. Box plot
    data['month'] = data['date'].dt.strftime('%b')
    fig.add_trace(
        go.Box(x=data['month'], y=data['sales'], name="Monthly Distribution",
               marker_color='#d62728'),
        row=2, col=2
    )

    # Update layout with properly positioned annotations
    fig.update_layout(
        height=1200,  # Increased height for better spacing
        width=1200,
        showlegend=True,
        title_text="Sales Analytics Dashboard",
        title_x=0.5,
        template='plotly_white',
        font=dict(family="Arial", size=10),
        margin=dict(t=100, b=100),  # Increase top and bottom margins
    )

    # Add annotations with proper positioning
    annotations = [
        # Title for the dashboard
        dict(
            text="Sales Analytics Dashboard",
            xref="paper", yref="paper",
            x=0.5, y=1.1,
            showarrow=False,
            font=dict(size=24)
        ),
        # Caption for Sales Trend
        dict(
            text="📈 The sales trend shows an upward trajectory with seasonal weekly patterns. " +
                 "The 7-day moving average helps identify the underlying growth trend.",
            xref="paper", yref="paper",
            x=0.25, y=0.57,  # Positioned below the first plot
            showarrow=False,
            font=dict(size=10, color="gray"),
            align="left"
        ),
        # Caption for Category Sales
        dict(
            text="📊 Category B consistently outperforms other categories, " +
                 "suggesting a strong market position and potential for expansion.",
            xref="paper", yref="paper",
            x=0.75, y=0.57,  # Positioned below the second plot
            showarrow=False,
            font=dict(size=10, color="gray"),
            align="left"
        ),
        # Caption for Summary Table
        dict(
            text="📋 Monthly summary statistics reveal seasonal patterns " +
                 "in both sales and temperature, with peak performance in summer months.",
            xref="paper", yref="paper",
            x=0.25, y=0.02,  # Positioned below the table
            showarrow=False,
            font=dict(size=10, color="gray"),
            align="left"
        ),
        # Caption for Box Plot
        dict(
            text="📦 Box plot demonstrates higher sales volatility during summer months, " +
                 "with increased outliers and wider distributions.",
            xref="paper", yref="paper",
            x=0.75, y=0.02,  # Positioned below the box plot
            showarrow=False,
            font=dict(size=10, color="gray"),
            align="left"
        )
    ]

    fig.update_layout(annotations=annotations)

    # Update margins for subplots to accommodate annotations
    fig.update_layout(
        xaxis1=dict(domain=[0, 0.45]),
        xaxis2=dict(domain=[0.55, 1]),
        xaxis3=dict(domain=[0, 0.45]),
        xaxis4=dict(domain=[0.55, 1])
    )

    return fig

def export_dashboard(fig, filename="dashboard.pdf"):
    """Export the dashboard to PDF"""
    fig.write_image(filename, engine="kaleido")

# Create and export the dashboard
data = create_sample_data()
dashboard = create_dashboard(data)

# Export to PDF
export_dashboard(dashboard)

C:\Users\mrasm\AppData\Local\Temp\ipykernel_17200\3773775506.py:25: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



PermissionError: [Errno 13] Permission denied: 'dashboard.pdf'

In [16]:
# Do all analysis and get plots
# Add this before calling fred_visualize
display(df.head())


current_plot, predictive_plot = fred_visualize(df)

# Show results
current_plot.show()
predictive_plot.show()

,yield_spread,gdp,fed_funds,unemployment,option_adjusted_spread,delinquency_rate_credit_cards,delinquency_rate_loans,cpi,pce,gdp_growth,quarterly_spread,next_quarter_delinquency,quarter,period
date,,,,,,,,,,,,,,
1976-01-01,NaN,6323.649,5.37,7.9,NaN,NaN,NaN,55.8,27.795,NaN,NaN,NaN,1Q76,Expansion
1976-01-02,NaN,NaN,5.28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1Q76,Expansion
1976-01-03,NaN,NaN,5.28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1Q76,Expansion
1976-01-04,NaN,NaN,5.28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1Q76,Expansion
1976-01-05,NaN,NaN,5.29,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1Q76,Expansion


Data prepared. Shape: (222, 5)
Columns after preparation: ['quarter', 'period', 'quarterly_spread', 'next_quarter_delinquency', 'delinquency_rate_loans']
Credit Risk Analysis Results:


In [ ]:
# 1. First create a clean dataframe with current spreads and NEXT quarter's delinquencies
def create_prediction_df(df):
    prediction_df = pd.DataFrame({
        'Quarter': df['quarter'],
        'Current_Spread': df['quarterly_spread'],
        'Next_Quarter_Delinquency': df['delinquency_rate_loans'].shift(-1)  # Next quarter's delinquency
    })  # Remove any rows with missing data

    # Calculate correlation
    correlation = prediction_df['Current_Spread'].corr(prediction_df['Next_Quarter_Delinquency'])

    return prediction_df, correlation

# 2. Create scatter plot to visualize relationship
def plot_prediction_relationship(prediction_df):
    """
    Create scatter plot showing the predictive relationship between spreads and future delinquencies.
    """
    # Calculate correlation for the title
    correlation = prediction_df['Current_Spread'].corr(prediction_df['Next_Quarter_Delinquency'])

    # Create the base scatter plot
    fig = px.scatter(
        prediction_df,
        x='Current_Spread',
        y='Next_Quarter_Delinquency',
        trendline="ols"
    )

    # Update scatter points
    fig.update_traces(
        marker=dict(
            size=12,
            color='#2E86C1',  # Rich blue
            line=dict(width=1, color='white')
        ),
        selector=dict(mode='markers')
    )

    # Style the trendline
    fig.update_traces(
        line=dict(width=2, dash='dot'),
        selector=dict(mode='lines')
    )

    # Enhanced layout
    fig.update_layout(
        title=dict(
            text=f'Predictive Credit Risk Analysis<br><span style="font-size: 14px">Next Quarter Delinquency vs Current Spread (Correlation: {correlation:.3f})</span>',
            font=dict(size=24),
            x=0.5,
            y=0.95
        ),
        plot_bgcolor='white',
        paper_bgcolor='white',
        font=dict(
            family="Arial",
            size=12
        ),
        width=1000,
        height=600,
        margin=dict(t=100, l=80, r=40, b=80)
    )

    # Enhance axes
    fig.update_xaxes(
        title=dict(
            text="Current Quarter Spread (basis points)",
            font=dict(size=14)
        ),
        showgrid=True,
        gridwidth=1,
        gridcolor='#E5E5E5',
        mirror=True,
        showline=True,
        linewidth=2,
        linecolor='#2C3E50'
    )

    fig.update_yaxes(
        title=dict(
            text="Next Quarter Delinquency Rate (%)",
            font=dict(size=14)
        ),
        showgrid=True,
        gridwidth=1,
        gridcolor='#E5E5E5',
        mirror=True,
        showline=True,
        linewidth=2,
        linecolor='#2C3E50'
    )

    # Add annotations
    fig.add_annotation(
        text="Source: Federal Reserve Economic Data (FRED)",
        xref="paper", yref="paper",
        x=0.0000001, y=-0.15,
        showarrow=False,
        font=dict(size=9, color='gray')
    )

    # Add a subtle caption explaining the relationship
    fig.add_annotation(
        text="Higher spreads in the current quarter tend to predict increased delinquency rates in the following quarter",
        xref="paper", yref="paper",
        x=0.8, y=-0.15,
        showarrow=False,
        font=dict(size=10, color='gray')
    )

    return fig
# Run analysis
prediction_df, correlation = create_prediction_df(df_viz)
display(prediction_df)
print(f"Correlation between current spreads and next quarter's delinquencies: {correlation:.3f}")
plot_prediction_relationship(prediction_df)

,Quarter,Current_Spread,Next_Quarter_Delinquency
7670,4Q96,3.130000,1.90
7671,1Q97,2.897377,1.81
7761,2Q97,2.747231,1.72
7852,3Q97,2.620462,1.70
7944,4Q97,2.780606,1.81
...,...,...,...
17440,4Q23,3.978000,1.12
17532,1Q24,3.365000,1.13
17623,2Q24,3.180152,1.13
17714,3Q24,3.259851,1.13


Correlation between current spreads and next quarter's delinquencies: 0.704


In [ ]:
def analyze_spread_signals(df):
    """
    Analyze the relationship between spreads and delinquencies, create signals,
    and evaluate their accuracy.
    """
    # Calculate historical mean and create signals
    historical_mean = df['quarterly_spread'].mean()

    def interpret_spread_signal(current_spread, historical_mean):
        if current_spread > (historical_mean * 1.2):
            return "HIGH ALERT: Expect delinquencies to rise next quarter"
        elif current_spread > (historical_mean * 1.1):
            return "WATCH: Moderate risk of rising delinquencies"
        return "NORMAL: No significant increase in delinquencies expected"

    # Create analysis dataframe with signals and future delinquencies
    df['signal'] = df['quarterly_spread'].apply(
        lambda x: interpret_spread_signal(x, historical_mean)
    )
    df['next_quarter_delinq'] = df['delinquency_rate_loans'].shift(-1)
    df['delinquency_increased'] = df['next_quarter_delinq'] > df['delinquency_rate_loans']

    # Create visualization
    fig = px.scatter(df,
                    x='quarterly_spread',
                    y='delinquency_rate_loans',
                    color='signal',
                    hover_data=['quarter'],
                    title='Spread Levels and Their Signals',
                    labels={
                        'quarterly_spread': 'Spread (%)',
                        'delinquency_rate_loans': 'Delinquency Rate (%)'
                    })

    # Add threshold lines
    fig.add_vline(x=historical_mean * 1.2,
                  line_dash="dash",
                  annotation_text="High Alert Threshold",
                  line_color="red")
    fig.add_vline(x=historical_mean * 1.1,
                  line_dash="dash",
                  annotation_text="Watch Threshold",
                  line_color="yellow")

    # Calculate accuracy metrics
    signal_accuracy = df.groupby('signal')['delinquency_increased'].agg([
        'count',  # number of signals
        'mean',   # accuracy rate
        'sum'     # number of correct predictions
    ]).round(2)

    # Calculate crisis period accuracy (2008-2009)
    crisis_accuracy = df[df['quarter'].str.contains('2008|2009')].groupby('signal')['delinquency_increased'].mean()

    # Print analysis results
    print(f"\nHistorical Mean Spread: {historical_mean:.2f}%")
    print(f"Watch Threshold (110%): {historical_mean * 1.1:.2f}%")
    print(f"High Alert Threshold (120%): {historical_mean * 1.2:.2f}%")

    print("\nRecent Market Signals:")
    print(df[['quarter', 'quarterly_spread', 'signal']].tail())

    print("\nSignal Accuracy Analysis:")
    print(signal_accuracy)

    print("\nAccuracy During Financial Crisis (2008-2009):")
    print(crisis_accuracy)

    return fig

# Run the analysis
fig = analyze_spread_signals(df_viz)
fig.show()


Historical Mean Spread: 5.27%
Watch Threshold (110%): 5.80%
High Alert Threshold (120%): 6.33%

Recent Market Signals:
      quarter  quarterly_spread  \
17440    4Q23          3.978000   
17532    1Q24          3.365000   
17623    2Q24          3.180152   
17714    3Q24          3.259851   
17806    4Q24          2.908214   

                                                  signal  
17440  NORMAL: No significant increase in delinquenci...  
17532  NORMAL: No significant increase in delinquenci...  
17623  NORMAL: No significant increase in delinquenci...  
17714  NORMAL: No significant increase in delinquenci...  
17806  NORMAL: No significant increase in delinquenci...  

Signal Accuracy Analysis:
                                                    count  mean  sum
signal                                                              
HIGH ALERT: Expect delinquencies to rise next q...     30  0.47   14
NORMAL: No significant increase in delinquencie...     78  0.33   26
WATCH: Moder

In [ ]:
# Fifth Cell - Data Visualization

fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add spreads
fig.add_trace(
    go.Scatter(x=df_viz['quarter'], y=df_viz['quarterly_spread'], name="Option-Adjusted Spread %"),
    secondary_y=False
)

# Add delinquency
fig.add_trace(
    go.Scatter(x=df_viz['quarter'], y=df_viz['delinquency_rate_loans'], name="Delinquency Rate Loan %"),
    secondary_y=True
)

# Update layout
fig.update_layout(
    title='Option-Adjusted Spread vs Corporate Loan Delinquency'
)

fig.update_yaxes(title_text="Option-Adjusted Spread (%)", secondary_y=False)
fig.update_yaxes(title_text="Delinquency Rate (%)", secondary_y=True)

fig.show()
pio.write_image(fig, "data_viz.pdf", format="pdf", engine="kaleido")

fig = go.Figure(data=[go.Table(
    header=dict(values=list(df_viz.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[df_viz[col] for col in df_viz.columns],
               fill_color='lavender',
               align='left'))
])

fig.show()